### Supress Warnings and import all the relevant packages and libraries

In [ ]:
# Supress Warnings

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import the necessary packages and libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 50)

from sklearn.model_selection import train_test_split

plt.style.use('ggplot')

### Reading and preparing the data

In [ ]:
# Read the data

bike = pd.read_csv('/kaggle/input/boombikes/day.csv')
bike.head()

In [ ]:
# Checking the shape of the dataframe

bike.shape

In [ ]:
# Checking the info for all the columns

bike.info()

In [ ]:
# Dropping the unnecessary columns

bike = bike.drop(['instant','dteday','casual','registered','atemp'], axis=1)
bike.shape

In [ ]:
# Renaming the columns for better understanding

bike.rename(columns = {'yr':'year','mnth':'month','hum':'humidity','cnt':'count'}, inplace = True) 
bike.head()

### Identifying the categorical and continuous variables

In [ ]:
bike.nunique().sort_values()

In [ ]:
# Mapping variables season, month, weekday, weathersit

bike['season'] = bike.season.map({1: 'spring', 2: 'summer',3:'fall', 4:'winter'})

bike['month'] = bike.month.map({1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'June', 7:'July', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'})

bike['weekday']  =bike.weekday.map({0:'Sun', 1:'Mon', 2:'Tue', 3:'Wed', 4:'Thu',  5:'Fri', 6:'Sat'})

bike['weathersit'] = bike.weathersit.map({1: 'Clear', 2:'Cloudy', 3:'Light Snow', 4:'Heavy Rain'})

bike.head()

### Visualising the numeric variables

In [ ]:
# Pairplot of all the numeric variables

sns.pairplot(bike, vars=['count', 'temp', 'humidity', 'windspeed'])
plt.show()

### Visualising the categorical variables

In [ ]:
#Boxplot for some of the categorical variables with respect to the target varibale 'count'

plt.figure(figsize=(20, 12))
plt.subplot(2,4,1)
sns.boxplot(x = 'year', y = 'count', data = bike)
plt.subplot(2,4,2)
sns.boxplot(x = 'holiday', y = 'count', data = bike)
plt.subplot(2,4,3)
sns.boxplot(x = 'workingday', y = 'count', data = bike)
plt.subplot(2,4,4)
sns.boxplot(x = 'weathersit', y = 'count', data = bike)
plt.subplot(2,4,5)
sns.boxplot(x = 'season', y = 'count', data = bike)
plt.subplot(2,4,6)
sns.boxplot(x = 'weekday', y = 'count', data = bike)
plt.subplot(2,4,7)
sns.boxplot(x = 'month', y = 'count', data = bike)
plt.show()

### Analysis between the target variable - count, and the other variables

In [ ]:
# Analysis between weathersit and count

plt.figure(figsize=(10,4))
sns.barplot('weathersit','count',data=bike)
plt.title('Bike Rentals in different Weather Situations',fontsize=12)
plt.show()

In [ ]:
# Analysis between season and count

plt.figure(figsize=(10,4))
sns.barplot('season','count',data=bike)
plt.title('Bike Rentals in different Seasons',fontsize=12)
plt.show()

In [ ]:
# Analysis of the Bike Rentals for each month of both the years

plt.figure(figsize=(10,5))
sns.barplot('month','count',hue='year',data=bike)
plt.title('Bike Rentals in different Months of both the Years',fontsize=12)
plt.show()

In [ ]:
# Analysis of Bike Rentals with Temperature

sns.scatterplot(x='temp',y='count' ,data=bike)
plt.title('Temp vs Count')
plt.show()

In [ ]:
# Analysis of Bike Rentals with Humidity

sns.scatterplot(x='humidity',y='count' ,data=bike)
plt.title('Humidity vs Count')
plt.show()

### Correlation between the variables

In [ ]:
# Heatmap to visualise the correlation between the variables

plt.figure(figsize=(10, 5))
sns.heatmap(bike.corr(), cmap="YlGnBu", annot = True)
plt.title("Correlation between the variables")
plt.show()

Both the pairplot and heatmap, above, help to interpret the data well and identify the variables that can turn out to be useful in building the model. Hence, Linear Regression Model can be used.

### Creating Dummy Variables

Creating dummy variables for - month, season, weathersit and weekday

In [ ]:
month_dummy = pd.get_dummies(bike.month,drop_first=True)
weekday_dummy = pd.get_dummies(bike.weekday,drop_first=True)
weathersit_dummy = pd.get_dummies(bike.weathersit,drop_first=True)
season_dummy = pd.get_dummies(bike.season,drop_first=True)

In [ ]:
# Adding the dummy variables to the original dataframe

bike = pd.concat([month_dummy,weekday_dummy,weathersit_dummy,season_dummy,bike],axis=1)
bike.head()

In [ ]:
# Dropping the original columns - month, weekday, weathersit and season, since dummy variables have already been created for them

bike.drop(['season','month','weekday','weathersit'], axis = 1, inplace = True)
bike.shape

### Splitting the Data into Training and Testing Sets

In [ ]:
np.random.seed(0)

bike_train, bike_test = train_test_split(bike, train_size = 0.7, random_state = 100)

print(bike_train.shape)
print(bike_test.shape)

### Rescaling the Features using MinMax Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
# Creating a list of numeric variables

num_vars=['temp','humidity','windspeed','count']

In [ ]:
# Fit the data

bike_train[num_vars] = scaler.fit_transform(bike_train[num_vars])
bike_train.head()

In [ ]:
# Checking numeric variables after scaling the features
bike_train.describe()

### Dividing into X and y sets for the model building

In [ ]:
y_train = bike_train.pop('count')
X_train = bike_train

### Building our model

We will be using the **LinearRegression function from SciKit Learn** for its compatibility with RFE - Recursive Feature Elimination

In [ ]:
# Importing RFE and LinearRegression

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [ ]:
# Running RFE with the output number of the variable equal to 15

lm = LinearRegression()
lm.fit(X_train, y_train)

# Running RFE
rfe = RFE(lm, 15)             
rfe = rfe.fit(X_train, y_train)

In [ ]:
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

In [ ]:
# Columns for which rfe_support is true

col = X_train.columns[rfe.support_]
col

### Building model using statsmodel, for the detailed statistics

### Model 1

In [ ]:
# Creating the first dataframe model with RFE selected variables
X_train_1 = X_train[col]

In [ ]:
# Adding a constant variable 

import statsmodels.api as sm  
X_train_1 = sm.add_constant(X_train_1)

In [ ]:
# Running the linear model

lm = sm.OLS(y_train,X_train_1).fit() 

In [ ]:
# Summary of our linear model
print(lm.summary())

In [ ]:
# Dropping the const variable

X_train_1 = X_train_1.drop(['const'], axis=1)

In [ ]:
# Calculating the VIFs for the new model

from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
X = X_train_1
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

We will be following the below rule to drop the variables one by one, as per the priorities mentioned by their sequences:-

* We will first check the summary and VIF
* If a variable has got high p-value(>0.05) as well as high VIF(>5), we need to drop that first
* If a variable has got high p-value(>0.05) but low VIF(<5), then we need to drop such
* Still if we have a variable with low p-value(<0.05) but high VIF(>5), we need to drop such at the very end

### Model 2

In [ ]:
# Rebuilding the model without 'Sat'

X_train_2 = X_train_1.drop(['Sat'], axis=1)

In [ ]:
# Adding the contsant variable

X_train_2 = sm.add_constant(X_train_2)

In [ ]:
# Running the linear model

lm = sm.OLS(y_train,X_train_2).fit() 

In [ ]:
# Summary of the new model
print(lm.summary())

In [ ]:
# Dropping the const variable

X_train_2 = X_train_2.drop(['const'], axis=1)

In [ ]:
# Calculating the VIFs for the new model

vif = pd.DataFrame()
X = X_train_2
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

### Model 3

In [ ]:
# Rebuilding the model without 'humidity'

X_train_3 = X_train_2.drop(['humidity'], axis=1)

In [ ]:
# Adding the contsant variable

X_train_3 = sm.add_constant(X_train_3)

In [ ]:
# Running the linear model

lm = sm.OLS(y_train,X_train_3).fit()

In [ ]:
# Summary of the new model
print(lm.summary())

In [ ]:
# Dropping the const variable

X_train_3 = X_train_3.drop(['const'], axis=1)

In [ ]:
# Calculating the VIFs for the new model

vif = pd.DataFrame()
X = X_train_3
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

### Model 4

In [ ]:
# Rebuilding the model without 'windspeed'

X_train_4 = X_train_3.drop(['windspeed'], axis=1)

In [ ]:
# Adding the contsant variable

X_train_4 = sm.add_constant(X_train_4)

In [ ]:
# Running the linear model

lm = sm.OLS(y_train,X_train_4).fit()

In [ ]:
# Summary of the new model
print(lm.summary())

In [ ]:
# Dropping the const variable

X_train_4 = X_train_4.drop(['const'], axis=1)

In [ ]:
# Calculating the VIFs for the new model

vif = pd.DataFrame()
X = X_train_4
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

### Model 5

In [ ]:
# Rebuilding the model without 'Jan'

X_train_5 = X_train_4.drop(['Jan'], axis=1)

In [ ]:
# Adding the contsant variable

X_train_5 = sm.add_constant(X_train_5)

In [ ]:
# Running the linear model

lm = sm.OLS(y_train,X_train_5).fit()

In [ ]:
# Summary of the new model
print(lm.summary())

In [ ]:
# Dropping the const variable

X_train_5 = X_train_5.drop(['const'], axis=1)

In [ ]:
# Calculating the VIFs for the new model

vif = pd.DataFrame()
X = X_train_5
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

### Model 6

In [ ]:
# Rebuilding the model without 'Dec'

X_train_6 = X_train_5.drop(['Dec'], axis=1)

In [ ]:
# Adding the contsant variable

X_train_6 = sm.add_constant(X_train_6)

In [ ]:
# Running the linear model

lm = sm.OLS(y_train,X_train_6).fit()

In [ ]:
# Summary of the new model
print(lm.summary())

In [ ]:
# Dropping the const variable

X_train_6 = X_train_6.drop(['const'], axis=1)


In [ ]:
# Calculating the VIFs for the new model

vif = pd.DataFrame()
X = X_train_6
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

### Model 7

In [ ]:
# Rebuilding the model without 'Nov'

X_train_7 = X_train_6.drop(['Nov'], axis=1)

In [ ]:
# Adding the contsant variable

X_train_7 = sm.add_constant(X_train_7)

In [ ]:
# Running the linear model

lm = sm.OLS(y_train,X_train_7).fit()

In [ ]:
# Summary of the new model
print(lm.summary())

In [ ]:
# Calculating the VIFs for the new model

vif = pd.DataFrame()
X = X_train_7
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif = vif[vif['Features']!='const']    # Ignoring to display the vif of 'const'
vif

### Residual Analysis of the train data

So, now to check if the error terms are also normally distributed (which is infact, one of the major assumptions of linear regression), let us plot the histogram of the error terms and see what it looks like.

In [ ]:
lm = sm.OLS(y_train,X_train_7).fit()  #As obtained previously
y_train_count = lm.predict(X_train_7)

In [ ]:
# Plot the histogram of the error terms

fig = plt.figure()
sns.distplot((y_train - y_train_count), bins = 20)
fig.suptitle('Error Terms', fontsize = 20)                  # Plot heading 
plt.xlabel('Errors', fontsize = 18)                         # X-label
plt.show()

### Applying the scaling on the test sets

In [ ]:
num_vars=['temp','humidity','windspeed','count']

# Fit and transform operations are done on the training data but only transform operation will be done on the test data

bike_test[num_vars] = scaler.transform(bike_test[num_vars])
bike_test.describe()

### Dividing into X_test and y_test

In [ ]:
y_test = bike_test.pop('count')
X_test = bike_test

In [ ]:
# Adding constant variable to test dataframe

X_test_m7 = sm.add_constant(X_test)

In [ ]:
# Creating X_test_m7 dataframe by dropping variables which were removed till our final Model 7 in the training dataset

X_test_m7 = X_test_m7.drop(['Aug','Feb','June','Mar','May','Oct','Mon','Sun','Thu','Tue','Wed','summer','workingday','Sat','humidity','windspeed','Jan','Dec','Nov'], axis = 1)

In [ ]:
# Making predictions using the seventh model

y_pred_m7 = lm.predict(X_test_m7)

### Model Evaluation

In [ ]:
# Plotting y_test and y_pred to understand the spread

fig = plt.figure()
plt.scatter(y_test, y_pred_m7)
fig.suptitle('y_test vs y_pred', fontsize = 20)              # Plot heading 
plt.xlabel('y_test', fontsize = 18)                          # X-label
plt.ylabel('y_pred', fontsize = 16)      
plt.show()

In [ ]:
# Regression plot

sns.regplot(x = y_test, y = y_pred_m7, fit_reg=True,scatter_kws={"color": "blue"}, line_kws={"color": "red"})

plt.title('y_test vs y_pred', fontsize=20)              # Plot heading 
plt.xlabel('y_test', fontsize=18)                          # X-label
plt.ylabel('y_pred', fontsize=16)                          # Y-label
plt.show()

### Calculation of R-square and Adjusted R-square values

In [ ]:
# Evaluate R-square for test dataset

from sklearn.metrics import r2_score
r2_score(y_test,y_pred_m7)

In [ ]:
# Adjusted R^2
# adj r2 = 1-((1-R2)*(n-1)/(n-p-1))

# n = sample size (in this case the value is 220, as yielded before)
# p = number of independent variables(in this case the value is 9)

Adj_r2 = 1 - ((1 - 0.8065842474886509) * 219 / (220-9-1))
print(Adj_r2)

For the training dataset, the R^2 value was 0.822 and adjusted R^2 value was 0.819.

For the testing dataset, the R^2 value obtained is 0.809 and adjusted R^2 value obtained is 0.798.




Hence the equation of our best fitted line is:-

$ count = 0.0654 \times Sep + 0.0534 \times winter + 0.2332 \times year + 0.4695 \times temp - 0.0690 \times July - 0.0781 \times Cloudy - 0.2993 \times Light Snow - 0.1122 \times spring - 0.1006 \times holiday $


Overall we have a decent model, but we also acknowledge that we could do better. 

### Interpretations

* We have arrived at a very decent model for the the demand for shared bikes with the significant variables.

* We can see that temperature variable is having the highest coefficient of 0.4695, which means if the temperature increases by one unit, the number of bike rentals increases by 0.4695 units.

* The other significant variables having positive coefficients are September and Winter.

* There are some variables with negative coefficients too, like July, Cloudy, Light Snow, Spring and Holiday. A negative coefficient suggests that, as the independent variable increases, the dependent variable tends to decrease,and vice-versa.

### Business Goals

* The demand for bikes is observed to be more in 2019, than its previous year 2018. The recent dip in business may be due to the ongoing pandemic situation.

* The demand for bike rentals are found to be the least during the Spring season. Hence, special discounts may be considered during this season to get back the demand.

* The demand for bike rentals are mostly on Saturdays, but very less on the Sundays, probably due to the weekend holiday.

* January and February are the months, in which demands have been observed to be at the bottom, whereas September and its adjacent months show the highest demands. This is probably due to the vacation time during those two impacted months.

* The rental demands are the most during the clear days. But sharp fall in the demand have been observed during the adverse weather conditions during snowing, raining and thunderstorms.

* Temparature plays a cruical role in the bike rental demand. The demand seems to be pretty decent on a day with a medium to high temperature. Lower the temperature, lower the demand.

* The company needs to provide special discounts to attract more rental demands during the Spring season, or during the months of January and February, on Sundays, or on a particular day when it is snowing or raining.